# REVVER (rev)

This challenge generates 20 pieces of random shellcode and asks you to submit answers that satisfy them. This seems a like massive task to write an automatic solver to solve arbitrary shellcode, so it's very likely that there's a small number of patterns that we can discern to simplify the problem.

To that end, the first step (not shown here) is to collect lots of [samples](revver1100.txt) (each binary has a label from "one" to "twenty") and see if there are any patterns. We quickly find that there is a grouping mod 5, so that items {one, six, eleven, sixteen} are all similar and so on. Furthermore, some of the shellcodes have a very high degree of constantness, e.g. items {two, seven, twelve, seventeen} have the same shellcode except for six bytes. We will talk about these a bit more later, but here's an example diff of two shellcodes with label seventeen:

![diff](revver17diff.png)

In any case, inspecting more closely shows that the grouping is there because there are four different ways of collecting input. One involves `sys_read` and another involves `sys_getcwd`, but I didn't inspect these any closer (or really, the other two at all) as I correctly assumed that they were not really used.

Here's a summary of what the table looks like (with a typo in "fiveteen"):

| input source | xor | crc | rc4 | add | cpy |
| :--: | :--: | :--: | :--: | :--: | :--: |
| sys_read | one | two | three | four | five |
| ??? | six | seven | eight | nine | ten |
| ??? | eleven | twelve | thirteen | fourteen | fiveteen |
| sys_getcwd | sixteen | seventeen | eighteen | nineteen | twenty |

What follows is a quick summary of what each type looks like (mod 5):
1. (XOR) These shellcodes differ in exactly 11 bytes. Ten of these are the ciphertext, and the other one is the byte to xor it with. We use `pwn.xor` to solve this.
2. (CRC) These shellcodes differ in exactly 6 bytes. Four of these are a CRC32 value, and the remaining two are equal and denote the length of the input text. This is the only column with a non-unique solution. We use some from [crc32 tools](https://github.com/theonlypwner/crc32) to help us reverse this. In particular, we fix the string to begin a constant prefix and brute force the last 6 characters so that they are all ASCII.
3. (RC4) These shellcodes have a constant size, but the location of where the data is jumps around for various reasons. But the main component is a length-17 ASCII key and a length-22 ciphertext. We use `ARC4` to decrypt the ciphertext.
4. (ADD) These shellcodes have varying size, depending on the length of the answer. The way it is represented is simple enough, just take the differences between 2-byte words to form the plaintext. The harder bit is extracting where this data is in the shellcode, but we use `b'66c7'` as a heuristic to split at it works pretty well.
5. (CPY) These shellcodes have varying size, but have the easiest solution in theory -- the plaintext is already there so just copy it into the answer! In practice, this is the column with the highest failure rate because this plaintext really jumps around in the shellcode and ends up in various different places. Our heuristic is a combination of splitting at NUL bytes, and splitting at `b'5e5bc3'`, then looking for strings without a space (since it's usually between two error messages or something, both of which are likely to have spaces). This works maybe more than 50% of the time, which is good enough for the purposes of this challenge.

So that's a quick summary of the techniques used, and the actual code follows below. In particular, there are a lot of hardcoded offsets, just because that's the nature of the problem.

In [1]:
from pwn import *
from Crypto.Cipher import ARC4
import crc32
crc32.init_tables(0xedb88320)
nums = 'BAD,one,two,three,four,five,six,seven,eight,nine,ten,eleven,twelve,thirteen,fourteen,fiveteen,sixteen,seventeen,eighteen,nineteen,twenty'.split(',')

In [2]:
def lil(bs):
    return int.from_bytes(bs, 'little')

def solvecrc(ln, tgt):
    prefix = b'a' * (ln - 6)
    for i in crc32.permitted_characters:
        for j in crc32.permitted_characters:
            tmp = crc32.calc(prefix + bytes([i,j]))
            rev = list(crc32.findReverse(tgt, tmp))[0]
            if all(x in crc32.permitted_characters for x in rev):
                test = prefix + bytes([i, j]) + bytes(rev)
                assert len(test)==ln
                assert(crc32.calc(test)==tgt)
                return test

def solve(n,bs):
    if n % 5 == 1: # XOR
        s = [76,79,1748,151][(n-1)//5]
        return xor(bs[s+59], bytes([bs[i] for i in range(s,s+40,4)]))
    
    if n % 5 == 2: # CRC
        s = [1595,1448,1892,258][(n-1)//5]
        ln = bs[s+8]
        target = lil(bs[s:s+4])
        return solvecrc(ln, target)
    
    if n % 5 == 3: # RC4
        a,b,c = [ [2067,2139,2148], [1879,1954,1963], [2368,2443,2452], [783,858,867] ][n//5]
        a,b,ln = lil(bs[a:a+2]),lil(bs[b:b+2]),bs[c]
        x,y = bs[a:].split(b'\0')[0], bs[b:b+22]
        return ARC4.new(x).decrypt(y)

    if n % 5 == 4: # ADD
        foo = [(lil(x[2:4]) if x[0]==0x45 else lil(x[5:7])) for x in bs.split(bytes.fromhex('66c7'))[1:]]
        if n == 14:
            foo = foo[2:]
        assert len(foo)%2==0
        cnt = len(foo)//2
        bar = [((b-a)%65536).to_bytes(2,'big') for a,b in zip(foo[:cnt],foo[cnt:])]
        return b''.join(bar)
    
    if n % 5 == 0: # CPY
        if n == 15:
            return next(x for x in bs.split(b'\0')[-9:] if b' ' not in x)
        spl = [b'swag.key\0', b'Error!\n\0', 0, bytes.fromhex('5e5bc3')][(n-1)//5]
        
        if n == 5 and bs.split(spl)[-1].split(b'\0')[0] == b'':
            return bs.split(b'\0')[-3]
        
        return bs.split(spl)[-1].split(b'\0')[0]

With the solve methods now implemented, all that's left to do is actually run this on the remote server. Since we have some probability of failing (usually in column five), we will just keep repeating until we win.

In [3]:
def trial():
    with remote('revver-01.hfsc.tf',3320) as sh:
        sh.sendline(b'2')
        sh.recvuntil(b'RANDOM')
        for _ in range(20):
            mmm = sh.recvline(False).decode().split()[-1]
            print(mmm)
            if mmm == 'FAIL':
                return False
            num = nums.index(mmm)
            code = bytes.fromhex(sh.recvline(False).decode())
            test=solve(num,code)
            sh.recvuntil(b'ANSWER')
            #print(f'{test=}')
            sh.sendline(test)
            sh.recvline()
        print(sh.recvall())
        return True
    
while not trial():
    pass

[x] Opening connection to revver-01.hfsc.tf on port 3320
[x] Opening connection to revver-01.hfsc.tf on port 3320: Trying 159.65.24.100
[+] Opening connection to revver-01.hfsc.tf on port 3320: Done
eleven
sixteen
eight
fourteen
nine
four
three
seventeen
fiveteen
five
six
eighteen
twelve
two
ten
FAIL
[*] Closed connection to revver-01.hfsc.tf port 3320
[x] Opening connection to revver-01.hfsc.tf on port 3320
[x] Opening connection to revver-01.hfsc.tf on port 3320: Trying 159.65.24.100
[+] Opening connection to revver-01.hfsc.tf on port 3320: Done
twenty
eleven
seven
ten
twelve
one
two
five
three
nineteen
fiveteen
fourteen
thirteen
seventeen
four
sixteen
eighteen
eight
six
nine
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 63B
[+] Receiving all data: Done (63B)
[*] Closed connection to revver-01.hfsc.tf port 3320
b" \nb'midnight{Reversing_got_alot_easier_with_backup_cameras}'\n \n"
